__Detects when a student has missed an assignment within n past assignments and then sends reminders__

* May 2023 Bug sent message to students who had done assignment when used bulk sender



In [ ]:
# TEST = True
TEST = False

SEND = True
# SEND = False

In [ ]:
# Dynamically sets location so do not have to manually toggle between dropbox and documents
import os
path = "/".join([a for a in os.path.abspath("").split('/') if a not in ['Notebooks', 'personal']])
%cd $path

from CanvasHacks import environment

# Widgets
# from CanvasHacks.Widgets.SkaaControls import multiple_unit_control
# from CanvasHacks.Widgets.LiveSelection import make_test_selector
# from CanvasHacks.Widgets.AssignmentSelection import make_unit_chooser, make_selection_button

# Repos
# from CanvasHacks.Repositories.overview import SkaaOverviewRepository

# Data store
# Only used as a convenient way to load relevant repositories 
from CanvasHacks.Displays.dashboard import ControlStore

from CanvasHacks.StudentNaggers.skaa_review import SkaaReviewNagger

# Late reviews

## Send for multiple units

2024-05-02 This seems to work. Though worried about unit 6.

In [ ]:
def run_review_nagger_on_units(unit_numbers):
    """Sends reminder message to students who have not yet done the 
    assigned review for each of the provided units
    :unit_numbers List of integers
    """
    control_store = ControlStore()
    for i in unit_numbers:
        control_store.load_unit(i)
        repo = control_store.skaa_repos[i]
        nagger = SkaaReviewNagger(repo, is_test=TEST, send=SEND)
        print(f"Nagging about late reviews for unit {i}. \nTEST is {TEST}. \nSEND is {SEND}")
        nagger.run()

In [ ]:
units = [1,2,3, 4, 5]
# units = [i for i in range(1,7)]

In [ ]:
run_review_nagger_on_units(units)

## Send for single unit

2023-12-12 There are 2 errors. It is sending to students who have already done the assignment. 

Seems to be concentrated on unit 6

It is also raising the following exception
`
    ---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In [27], line 2
      1 # Actually runs
----> 2 nagger.run()

File ~/Dropbox/CanvasHacks/CanvasHacks/StudentNaggers/skaa_review.py:71, in SkaaReviewNagger.run(self)
     68 print( "Going to nag {} students to turn in SKAA review".format( len( self.recipients ) ) )
     70 for cid, name in self.recipients:
---> 71     self.messenger.send_message_to_student( cid, name )
     73 log_nag_messages(activity=self.activity, list_of_sent_messages=self.messenger.sent, is_dry_run=self.is_test)

File ~/Dropbox/CanvasHacks/CanvasHacks/Messaging/nagging.py:138, in ReviewNonSubmittersMessaging.send_message_to_student(self, student_id, first_name)
    137 def send_message_to_student( self, student_id, first_name ):
--> 138     body = self.prepare_message( first_name, student_id )
    140     if self.send:
    141         msg = self.messenger.send( student_id=student_id, subject=self.subject, body=body )

File ~/Dropbox/CanvasHacks/CanvasHacks/Messaging/nagging.py:120, in ReviewNonSubmittersMessaging.prepare_message(self, student_name, student_id)
    115 def prepare_message( self, student_name, student_id ):
    116     data = {
    117         'name': student_name,
    118         'assignment_name': self.activity.name,
    119         'original_due_date': self.activity.string_due_date,
--> 120         'invite_sent_date' : self.get_review_invitation_date( student_id )
    121     }
    123     return REVIEW_NON_SUBMITTER_TEMPLATE.format( **data )

File ~/Dropbox/CanvasHacks/CanvasHacks/Messaging/nagging.py:134, in ReviewNonSubmittersMessaging.get_review_invitation_date(self, student_id)
    132 # todo error handling needs to be added here so just returns empty string
    133 invite = self.invite_status_repo.get(student_id)
--> 134 return self.make_date_for_message(invite.sent_at)

AttributeError: 'NoneType' object has no attribute 'sent_at'`

In [ ]:
# make_test_selector()
make_unit_chooser(num_units=8)

In [ ]:
repo = control_store.skaa_repos[5]

# repo = control_store.skaa_repos[environment.CONFIG.unit.unit_number]
nagger = SkaaReviewNagger(repo, is_test=TEST, send=SEND)
print(f"Nagging about late reviews for unit {environment.CONFIG.unit.unit_number}. \nTEST is {TEST}. \nSEND is {SEND}")


In [ ]:
# Actually runs
nagger.run()

# Late essay assignments

Fall 2023: Accidentally sent reminder to at least one student who had turned in the previous week.

In [ ]:
from CanvasHacks.StudentNaggers.essay import EssayNagger

def run_essay_nagger_on_units(unit_numbers, test=TEST, send=SEND):
    """Sends reminder message to students who have not yet done the 
    assigned essay for each of the provided units
    :unit_numbers List of integers
    """
    # not using its log display features, just a convenient way to load what's needed. 
    control_store = ControlStore()
    for i in unit_numbers:
        control_store.load_unit(i)
        repo = control_store.skaa_repos[i]
        nagger = EssayNagger(repo, is_test=test, send=send)
        print(f"Nagging about late reviews for unit {i}. \nTEST is {test}. \nSEND is {send}")
        nagger.run()

In [ ]:
# units = [1]
units = [i for i in range(1,5)]
run_essay_nagger_on_units(units)

# Consolidated controls (dev)

In [ ]:
from CanvasHacks.Widgets.NagControls import nag_button_area

store = ControlStore()
nag_button_area(store)